In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare

# Chi Square Test

I suspect the dice in my Settlers of Catan set are not fair dice. So I wrote down all the outcomes of two games and I'm going to do a Chi Square test for goodness of fit.

# Load the data

In [2]:
df = pd.read_csv('../data/catan_dice.csv')

In [3]:
df.head(1)

,Value,Count G1,Count G2
0,2,2,1


# Set index to roll outcomes

Where `G1` is 'game 1' and `G2` is 'game 2'.

In [4]:
df = pd.DataFrame(df[['Count G1', 'Count G2']].values, index=df['Value'], columns=['G1', 'G2'])

In [5]:
df.head()

,G1,G2
Value,,
2,2,1
3,2,2
4,5,0
5,5,10
6,13,7


# Roll outcomes for each game and sum of both

In [6]:
df['Both'] = df['G1'] + df['G2']

In [7]:
df

,G1,G2,Both
Value,,,
2,2,1,3
3,2,2,4
4,5,0,5
5,5,10,15
6,13,7,20
7,6,3,9
8,6,5,11
9,4,1,5
10,7,2,9


# Find the observed totals

In [8]:
outcomes_dict = {}

for col in df.columns:
    outcomes_dict[col] = df[col].sum()

print('Total Rolls')

for i, j in outcomes_dict.items():
    print(f'{i:-<10} {j}')

Total Rolls
G1-------- 52
G2-------- 31
Both------ 83


# Get expected probabilities for dice rolls

In [9]:
from collections import defaultdict

expected = defaultdict(float)

In [10]:
rolls = [i for i in range(1,7)]
rolls

[1, 2, 3, 4, 5, 6]

In [11]:
for i in range(1,13):
    for j in rolls:
        for k in rolls:
            if j+k == i:
                expected[i] += 1/36

In [12]:
print('Probability of the sum of two dice')
for i, j in expected.items():
    print(f'{i:-<5} {j:.3}')

Probability of the sum of two dice
2---- 0.0278
3---- 0.0556
4---- 0.0833
5---- 0.111
6---- 0.139
7---- 0.167
8---- 0.139
9---- 0.111
10--- 0.0833
11--- 0.0556
12--- 0.0278


**And we put the values in a `np.array`**

In [13]:
e = np.array(list(expected.values()))
print(e[:3], '...')

[ 0.02777778  0.05555556  0.08333333] ...


# Find expected outcomes based on total rolls

In [14]:
for col in df.columns:
    # use outcomes_dict
    ex = outcomes_dict[col]*e
    df[f'e_{col}'] = ex
    print(ex[:3], '...\n')

[ 1.44444444  2.88888889  4.33333333] ...

[ 0.86111111  1.72222222  2.58333333] ...

[ 2.30555556  4.61111111  6.91666667] ...



In [15]:
df

,G1,G2,Both,e_G1,e_G2,e_Both
Value,,,,,,
2,2,1,3,1.444444,0.861111,2.305556
3,2,2,4,2.888889,1.722222,4.611111
4,5,0,5,4.333333,2.583333,6.916667
5,5,10,15,5.777778,3.444444,9.222222
6,13,7,20,7.222222,4.305556,11.527778
7,6,3,9,8.666667,5.166667,13.833333
8,6,5,11,7.222222,4.305556,11.527778
9,4,1,5,5.777778,3.444444,9.222222
10,7,2,9,4.333333,2.583333,6.916667


## Chi Square test

    chisq, p = scipy.stats.chisquare(f_obs, f_exp=None, ddof=0, axis=0)

[Citation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html)

In [16]:
d = {
    'G1': [i for i in df.columns if 'G1' in i],
    'G2': [i for i in df.columns if 'G2' in i],
    'Both': [i for i in df.columns if 'Both' in i]
}
d

{'Both': ['Both', 'e_Both'], 'G1': ['G1', 'e_G1'], 'G2': ['G2', 'e_G2']}

In [17]:
for i, j in d.items():
    obs = df[j[0]].values
    exp = df[j[1]].values
    chi, p = chisquare(obs, exp, ddof=1)
    print(f'{"Group:": <20}{i}')
    print(f'{"Chisq:": <20}{chi:.4}')
    print(f'{"P:": <20}{p:.4}')
    print('Observed/Expected pairs:')
    pairs = [(i,j,k) for i, j, k in zip(df.index, obs, exp)]
    for i, j, k in pairs:
        print(f'{i: <3} | {j: <2} / {k:.4}')
    print(f'{"":->30}')

Group:              G1
Chisq:              9.904
P:                  0.3583
Observed/Expected pairs:
2   | 2  / 1.444
3   | 2  / 2.889
4   | 5  / 4.333
5   | 5  / 5.778
6   | 13 / 7.222
7   | 6  / 8.667
8   | 6  / 7.222
9   | 4  / 5.778
10  | 7  / 4.333
11  | 1  / 2.889
12  | 1  / 1.444
------------------------------
Group:              G2
Chisq:              22.28
P:                  0.008022
Observed/Expected pairs:
2   | 1  / 0.8611
3   | 2  / 1.722
4   | 0  / 2.583
5   | 10 / 3.444
6   | 7  / 4.306
7   | 3  / 5.167
8   | 5  / 4.306
9   | 1  / 3.444
10  | 2  / 2.583
11  | 0  / 1.722
12  | 0  / 0.8611
------------------------------
Group:              Both
Chisq:              18.51
P:                  0.02971
Observed/Expected pairs:
2   | 3  / 2.306
3   | 4  / 4.611
4   | 5  / 6.917
5   | 15 / 9.222
6   | 20 / 11.53
7   | 9  / 13.83
8   | 11 / 11.53
9   | 5  / 9.222
10  | 9  / 6.917
11  | 1  / 4.611
12  | 1  / 2.306
------------------------------


# Conclusion

I don't like the look of this. Game one was ok. But Game two was absolutely a fluke! (If we assume the dice are fair.) But let's say there were too few rolls in game two. When we combine the games we still get something I don't quite like.

If we assume the dice are fair, we have a 2% chance that the observed distribution is at least this far from the expected distribution. Ok 2% is not so bad... but I lost game two because all of my opponents were on hexes with 5s on them and they all did better than me so... I conclude that the game was rigged in their favor.